# API Example
Hello Joel, rather than a task here is something I thought might be useful. This is an example of the requests task I told you to do with varying levels of quality.

In [7]:
import requests

url = "https://jsonplaceholder.typicode.com/posts"
response = requests.get(url)
data = response.json()
print(data)

[{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}, {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'}, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}, {'userId': 1, 'id': 4, 'title': 'eum et est occaecati', 'body': 'ullam et saepe reiciendis voluptatem adipisci\nsit amet autem assumenda provident rerum culpa\nquis hic c

In [13]:
import requests

# Note the incorrect URL
url = "https://jsonplaceholder.typicod.com/posts"
response = requests.get(url)
data = response.json()
print(data)

ConnectionError: HTTPSConnectionPool(host='jsonplaceholder.typicod.com', port=443): Max retries exceeded with url: /posts (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112b0ba90>: Failed to resolve 'jsonplaceholder.typicod.com' ([Errno 8] nodename nor servname provided, or not known)"))

Oh no! Our code didn't work when a single string was slightly wrong. So, the first thing to cover off on our journey to a production quality example would be -- what if something goes wrong? The network connection dies?, the computer crashs, the API throttles you?. So to this end we can use python's try/catch syntax.

In [22]:
def get_posts():
    url = "https://jsonplaceholder.typicode.com/posts"
    try:
        response = requests.get(url)
        response.raise_for_status() # So once we get our response we have access to response, we can use this to raise if there are any errors
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        # this is saying if there is a SPECIFIC error, here one causes by the Requests library, we will print the error
        # to test this, change a line in the URL so that is is wrong
        print(f"Error fetching data: {e}")
        return None

posts = get_posts()
if posts:
    print(f"Retrieved {len(posts)} posts")
    for post in posts[:5]:
        print(f"Post {post['id']}: {post['title']}")

Retrieved 100 posts
Post 1: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Post 2: qui est esse
Post 3: ea molestias quasi exercitationem repellat qui ipsa sit aut
Post 4: eum et est occaecati
Post 5: nesciunt quas odio


Ok so its getting better. We now can handle the scenario where the function fails. However our function isn't very usable. For example if we want to query a different endpoint it doesn't work. Also we don't know anything about the data once we have it -- is it a list?, a dict?, a scared little boy longing for friendship in a hostile world?. So here we can make use of more of the built in libraries (here `os, json` and `typing`) to give our code a bit of structure.

In [23]:
import requests
import os
import json
from typing import Any

# Lets us break down this type a bit since its a bit complicated. Starting in the inner layer we have:
# dict[str, Any] -> this means it is a dictionary where the key is a string, and the value is anything. E.g.,:
# {"data": "stuff"} or {"data": ["list", "of", "stuff"]} or {"data": 1} 
#
# Then that is wrapped in a list which just means we get back a list of the above dicts, like:
# [{"data": "stuff"} , {"data": "stuff"} , {"data": "stuff"}]
#
# Finally the '| None' part means that it could be the above or nothing. So formally the | (pipe) is Python's
# binary OR operator.

# Note: in older versions of python you might see the return signature as: Optional[List[Dict[str, Any]]]
def fetch_api_data(endpoint: str) -> list[dict[str, Any]] | None:
    """
    Fetch data from the JSONPlaceholder API.
    
    Args:
        endpoint: The API endpoint to fetch data from
        
    Returns:
        List of dictionaries containing the API response data, or None if an error occurred
    """
    base_url = os.getenv("API_BASE_URL", "https://jsonplaceholder.typicode.com")
    full_url = f"{base_url}/{endpoint}"
    
    try:
        response = requests.get(full_url, timeout=10)
        response.raise_for_status()
        return response.json()
    # We are now handling more specific errors and returning them to the user. This is much better as the user of this
    # fn can decide what to do. Maybe on a Timeout error they want to wait 5 seconds and try again, but on a HTTP 
    # one they want to reach out to the API maintainer and say something is wrong
    except requests.exceptions.Timeout:
        print("Request timed out. Please try again later.")
        return None
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except json.JSONDecodeError:
        print("Error decoding JSON response")
        return None


posts = fetch_api_data("posts")
if posts:
    print(f"Retrieved {len(posts)} posts")
    for post in posts[:3]:
        print(f"Post {post['id']}: {post['title']}")
        print(f"Content: {post['body'][:100]}...")
        print("-" * 50)

Retrieved 100 posts
Post 1: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Content: quia et suscipit
suscipit recusandae consequuntur expedita et cum
reprehenderit molestiae ut ut quas...
--------------------------------------------------
Post 2: qui est esse
Content: est rerum tempore vitae
sequi sint nihil reprehenderit dolor beatae ea dolores neque
fugiat blanditi...
--------------------------------------------------
Post 3: ea molestias quasi exercitationem repellat qui ipsa sit aut
Content: et iusto sed quo iure
voluptatem occaecati omnis eligendi aut ad
voluptatem doloribus vel accusantiu...
--------------------------------------------------


Next up we can see what happens when we extend our error typing to be able to handle specific instances of errors we define. This can be super useful when we are creating our own tools.

In [24]:
import json
import logging
import os
from typing import Any

import requests

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

class APIError(Exception):
    """Base exception for API-related errors."""
    pass

class APIConnectionError(APIError):
    """Exception raised when connection to the API fails."""
    pass

class APIResponseError(APIError):
    """Exception raised when the API returns an error response."""
    pass

def fetch_api_data(
    endpoint: str,
    params: dict[str, str | int] | None = None
) -> list[dict[str, Any]]:
    """
    Fetch data from the JSONPlaceholder API.
    
    Args:
        endpoint: The API endpoint to fetch data from
        params: Optional query parameters to include in the request
        
    Returns:
        List of dictionaries containing the API response data
        
    Raises:
        APIConnectionError: If there's an issue connecting to the API
        APIResponseError: If the API returns an error response
    """
    base_url = os.getenv("API_BASE_URL", "https://jsonplaceholder.typicode.com")
    full_url = f"{base_url}/{endpoint}"
    
    try:
        logger.debug(f"Making GET request to {full_url}")
        response = requests.get(full_url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        logger.error("Request timed out")
        raise APIConnectionError("Request timed out. Please try again later.")
    except requests.exceptions.HTTPError as e:
        logger.error(f"HTTP Error: {e}")
        raise APIResponseError(f"HTTP Error: {e}")
    except requests.exceptions.ConnectionError as e:
        logger.error(f"Connection Error: {e}")
        raise APIConnectionError(f"Connection Error: {e}")
    except requests.exceptions.RequestException as e:
        logger.error(f"Request Exception: {e}")
        raise APIConnectionError(f"Error fetching data: {e}")
    except json.JSONDecodeError:
        logger.error("Error decoding JSON response")
        raise APIResponseError("Error decoding JSON response")

def get_posts(limit: int | None = None) -> list[dict[str, Any]]:
    """
    Get posts from the API.
    
    Args:
        limit: Optional maximum number of posts to retrieve
        
    Returns:
        List of post dictionaries
    """
    params = {}
    if limit is not None:
        params["_limit"] = limit

    try:
        return fetch_api_data("posts", params) # type: ignore
    except APIError as e:
        logger.error(f"Error retrieving posts: {e}")
        return []

logger.setLevel(logging.INFO)

try:
    posts = get_posts(limit=5)
    
    if not posts:
        logger.warning("No posts were retrieved")
    else:
        logger.info(f"Retrieved {len(posts)} posts")
        for post in posts:
            print(f"Post {post['id']}: {post['title']}")
            print(f"Content: {post['body'][:100]}...")
            print("-" * 50)
except Exception as e:
    logger.exception(f"Unexpected error: {e}")

2025-04-06 10:51:39,903 - __main__ - INFO - Retrieved 5 posts


Post 1: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Content: quia et suscipit
suscipit recusandae consequuntur expedita et cum
reprehenderit molestiae ut ut quas...
--------------------------------------------------
Post 2: qui est esse
Content: est rerum tempore vitae
sequi sint nihil reprehenderit dolor beatae ea dolores neque
fugiat blanditi...
--------------------------------------------------
Post 3: ea molestias quasi exercitationem repellat qui ipsa sit aut
Content: et iusto sed quo iure
voluptatem occaecati omnis eligendi aut ad
voluptatem doloribus vel accusantiu...
--------------------------------------------------
Post 4: eum et est occaecati
Content: ullam et saepe reiciendis voluptatem adipisci
sit amet autem assumenda provident rerum culpa
quis hi...
--------------------------------------------------
Post 5: nesciunt quas odio
Content: repudiandae veniam quaerat sunt sed
alias aut fugiat sit autem sed est
voluptatem omnis possimus ess...
---

So now we want to move onto an example you would expect to see in more production-like code. The following is likely better than a lot of places. The key additions are:

- We use a dataclass for the APIConfig. Dataclasses are a special type of class in python that only holds data. This is in contrast with the standard ones that hold both data and functions. You can contrast APIConfig vs. APIClient
- We are using a class based design to group behaviours into objects. This is pretty typical in Python
- We are using some decorators (the things with @ above functions). These are basically functions that take in other functions.
- Finally we write some generic code. This is a bit beyond basic stuff so I recommend here you ask the Robot to explain to you how this example works :)

In [25]:
import json
import logging
import os
from dataclasses import dataclass
from typing import Any, Callable, Generic, TypeVar

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

T = TypeVar('T')

class APIError(Exception):
    """Base exception for API-related errors."""
    pass

class APIConnectionError(APIError):
    """Exception raised when connection to the API fails."""
    pass

class APIResponseError(APIError):
    """Exception raised when the API returns an error response."""
    def __init__(self, message: str, status_code: int | None = None):
        self.status_code = status_code
        super().__init__(message)

class APITimeoutError(APIConnectionError):
    """Exception raised when the API request times out."""
    pass

@dataclass
class APIConfig:
    """Configuration for the API client."""
    base_url: str
    timeout: int = 10
    max_retries: int = 3
    retry_backoff_factor: float = 0.5

class APIClient:
    """Client for interacting with external APIs."""
    
    def __init__(self, config: APIConfig | None = None):
        """
        Initialize the API client.
        
        Args:
            config: Configuration for the API client
        """
        self.config = config or APIClient.get_default_config()
        self.session = self._create_session()
    
    @staticmethod
    def get_default_config() -> APIConfig:
        """Get the default API configuration."""
        return APIConfig(
            base_url=os.getenv("API_BASE_URL", "https://jsonplaceholder.typicode.com"),
            timeout=int(os.getenv("API_TIMEOUT", "10")),
            max_retries=int(os.getenv("API_MAX_RETRIES", "3")),
            retry_backoff_factor=float(os.getenv("API_RETRY_BACKOFF", "0.5"))
        )
    
    def _create_session(self) -> requests.Session:
        """Create a requests session with retry configuration."""
        session = requests.Session()
        retry_strategy = Retry(
            total=self.config.max_retries,
            backoff_factor=self.config.retry_backoff_factor,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["GET"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        return session
    
    def get(
        self, 
        endpoint: str, 
        params: dict[str, str | int] | None = None
    ) -> Any:
        """
        Make a GET request to the API.
        
        Args:
            endpoint: The API endpoint
            params: Optional query parameters
            
        Returns:
            The parsed JSON response
            
        Raises:
            APIConnectionError: If there's an issue connecting to the API
            APIResponseError: If the API returns an error response
        """
        url = f"{self.config.base_url}/{endpoint}"
        logger.debug(f"Making GET request to {url} with params {params}")
        
        try:
            response = self.session.get(
                url, 
                params=params, 
                timeout=self.config.timeout
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.Timeout:
            logger.error("Request timed out")
            raise APITimeoutError("Request timed out. Please try again later.")
        except requests.exceptions.HTTPError as e:
            status_code = e.response.status_code if e.response else None
            logger.error(f"HTTP Error: {e}, Status Code: {status_code}")
            raise APIResponseError(f"HTTP Error: {e}", status_code=status_code)
        except requests.exceptions.ConnectionError as e:
            logger.error(f"Connection Error: {e}")
            raise APIConnectionError(f"Connection Error: {e}")
        except requests.exceptions.RequestException as e:
            logger.error(f"Request Exception: {e}")
            raise APIConnectionError(f"Error fetching data: {e}")
        except json.JSONDecodeError:
            logger.error("Error decoding JSON response")
            raise APIResponseError("Error decoding JSON response")

class Repository(Generic[T]):
    """Base repository class for handling API data."""
    
    def __init__(self, client: APIClient, endpoint: str, transformer: Callable[[dict[str, Any]], T]):
        """
        Initialize the repository.
        
        Args:
            client: The API client to use
            endpoint: The API endpoint for this repository
            transformer: A function to transform raw API data to the desired type
        """
        self.client = client
        self.endpoint = endpoint
        self.transformer = transformer
    
    def get_all(self, params: dict[str, Any] | None = None) -> list[T]:
        """
        Get all items from the API.
        
        Args:
            params: Optional query parameters
            
        Returns:
            List of transformed items
        """
        raw_data = self.client.get(self.endpoint, params)
        return [self.transformer(item) for item in raw_data]
    
    def get_by_id(self, item_id: int) -> T:
        """
        Get a single item by ID.
        
        Args:
            item_id: The ID of the item to retrieve
            
        Returns:
            The transformed item
            
        Raises:
            APIResponseError: If the item with the given ID doesn't exist
        """
        raw_data = self.client.get(f"{self.endpoint}/{item_id}")
        return self.transformer(raw_data)

@dataclass
class Post:
    """Represents a blog post."""
    id: int
    user_id: int
    title: str
    body: str
    
    @classmethod
    def from_api(cls, data: dict[str, Any]) -> 'Post':
        """Create a Post instance from API data."""
        return cls(
            id=data["id"],
            user_id=data["userId"],
            title=data["title"],
            body=data["body"]
        )

class PostRepository(Repository[Post]):
    """Repository for handling Post data."""
    
    def __init__(self, client: APIClient):
        """Initialize the PostRepository with an API client."""
        super().__init__(client, "posts", Post.from_api)
    
    def get_by_user_id(self, user_id: int) -> list[Post]:
        """
        Get all posts by a specific user.
        
        Args:
            user_id: The ID of the user
            
        Returns:
            List of Post objects
        """
        return self.get_all({"userId": user_id})

config = APIConfig(
    base_url="https://jsonplaceholder.typicode.com",
    timeout=5
)
client = APIClient(config)
post_repo = PostRepository(client)

posts = post_repo.get_all({"_limit": 5})
logger.info(f"Retrieved {len(posts)} posts")

for post in posts:
    print(f"ID: {post.id}, User ID: {post.user_id}")
    print(f"Title: {post.title}")
    print(f"Content preview: {post.body[:50]}...")
    print("-" * 50)

user_posts = post_repo.get_by_user_id(1)
logger.info(f"Retrieved {len(user_posts)} posts by user 1")

2025-04-06 10:51:40,492 - __main__ - INFO - Retrieved 5 posts
2025-04-06 10:51:40,681 - __main__ - INFO - Retrieved 10 posts by user 1


ID: 1, User ID: 1
Title: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Content preview: quia et suscipit
suscipit recusandae consequuntur ...
--------------------------------------------------
ID: 2, User ID: 1
Title: qui est esse
Content preview: est rerum tempore vitae
sequi sint nihil reprehend...
--------------------------------------------------
ID: 3, User ID: 1
Title: ea molestias quasi exercitationem repellat qui ipsa sit aut
Content preview: et iusto sed quo iure
voluptatem occaecati omnis e...
--------------------------------------------------
ID: 4, User ID: 1
Title: eum et est occaecati
Content preview: ullam et saepe reiciendis voluptatem adipisci
sit ...
--------------------------------------------------
ID: 5, User ID: 1
Title: nesciunt quas odio
Content preview: repudiandae veniam quaerat sunt sed
alias aut fugi...
--------------------------------------------------


There will be one more...